In [6]:
import pandas as pd
import numpy as np
import time
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    matthews_corrcoef,
    mean_absolute_error,
    mean_squared_error,
)
from sklearn.model_selection import train_test_split
import wittgenstein as lw

# Datei laden
print("Lade die CSV-Datei...")
csv_file_path = "C:\\Users\\jakob\\Documents\\Praktikum\\ML_Praktikum\\ml_praktikum_jagoetz_wkathari\\dataset\\clf_cat\\albert.csv"
data = pd.read_csv(csv_file_path)

# Zielvariable setzen (Spalte: "y")
target_column = "class"

if target_column not in data.columns:
    raise ValueError(f"Die Datei enthält keine '{target_column}'-Spalte. Bitte überprüfen.")

# Features und Zielvariable trennen
X = data.drop(columns=[target_column])
y = data[target_column]

# Prüfen, ob y bereits binär ist
unique_classes = np.unique(y)
print("Klassen in y:", unique_classes)
if len(unique_classes) != 2:
    raise ValueError("Die Zielvariable ist nicht binär, es werden zwei Klassen erwartet.")
    
# Überprüfen, ob die Klassen numerisch sind, wenn nicht, umwandeln:
if y.dtype == 'object':
    # Annahme: Hier liegen zwei Klassen vor, z.B. "default" und "no_default"
    # Alphabetisch sortieren:
    # Sofern nötig, kann man hier eine manuelle Zuordnung vornehmen.
    class_mapping = {cls: i for i, cls in enumerate(sorted(unique_classes))}
    y = y.map(class_mapping)

# Klassenverteilung ausgeben
print("Klassenverteilung:", np.unique(y, return_counts=True))

# Positive Klasse definieren (wir nehmen an, dass 1 die Default-Klasse ist)
pos_class = 1  # Falls nötig, sicherstellen, dass Klasse 1 tatsächlich der Default entspricht

# Diskretisierung der numerischen Features mit quantile
print("Diskretisiere numerische Features (quantile)...")
discretizer = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')
X_disc = discretizer.fit_transform(X)
X_disc = pd.DataFrame(X_disc, columns=X.columns)

# Anzahl der Wiederholungen
n_experiments = 10

# Matrizen für Ergebnisse
train_times = []
eval_times = []
accuracies = []
precisions = []
recalls = []
f1_scores = []
roc_aucs = []
mccs = []
maes = []
rmses = []

for i in range(n_experiments):
    print(f"\nStarte Experiment {i + 1}/{n_experiments} mit random_seed={i}...")

    # Stratifizierter Train-Test-Split
    X_train, X_test, y_train, y_test = train_test_split(X_disc, y, test_size=0.33, random_state=i, stratify=y)

    # RIPPER initialisieren
    ripper = lw.RIPPER(
        n_discretize_bins=10,
        verbosity=0,
    )

    # Trainingszeit messen
    start_time_train = time.time()
    ripper.fit(
        pd.concat([X_train, pd.Series(y_train, name=target_column)], axis=1),
        class_feat=target_column,
        pos_class=pos_class
    )
    end_time_train = time.time()
    train_time = end_time_train - start_time_train

    # Vorhersagen und Evaluierung
    start_time_eval = time.time()
    y_pred = ripper.predict(X_test)
    end_time_eval = time.time()
    eval_time = end_time_eval - start_time_eval

    # Berechnung der Metriken
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    mcc = matthews_corrcoef(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    # Für ROC AUC benötigen wir, falls möglich, eine probabilistische Vorhersage.
    # Falls nicht vorhanden, ist die binäre Vorhersage okay, aber der ROC_AUC ist dann begrenzt.
    # Hier verwenden wir die binären Vorhersagen:
    roc_auc = roc_auc_score(y_test, y_pred)

    # Ergebnisse speichern
    train_times.append(train_time)
    eval_times.append(eval_time)
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)
    mccs.append(mcc)
    maes.append(mae)
    rmses.append(rmse)
    roc_aucs.append(roc_auc)

    # Ergebnisse dieses Experiments ausgeben
    print(f"Trainingszeit: {train_time:.4f} s, Evaluierungszeit: {eval_time:.4f} s")
    print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}, ROC AUC: {roc_auc:.4f}")

# Ergebnisse in Matrizen umwandeln
train_times_matrix = np.array(train_times).reshape(1, -1)
eval_times_matrix = np.array(eval_times).reshape(1, -1)
metrics_matrix = {
    "Accuracy": np.array(accuracies).reshape(1, -1),
    "Precision": np.array(precisions).reshape(1, -1),
    "Recall": np.array(recalls).reshape(1, -1),
    "F1-Score": np.array(f1_scores).reshape(1, -1),
    "MCC": np.array(mccs).reshape(1, -1),
    "MAE": np.array(maes).reshape(1, -1),
    "RMSE": np.array(rmses).reshape(1, -1),
    "ROC AUC": np.array(roc_aucs).reshape(1, -1),
}

# Ausgabe der Ergebnisse
print("\n=== Ergebnis-Matrizen ===")
print("\nTrainingszeiten (s):")
print(train_times_matrix)

print("\nEvaluierungszeiten (s):")
print(eval_times_matrix)

print("\nMetriken:")
for metric_name, matrix in metrics_matrix.items():
    print(f"\n{metric_name}:")
    print(matrix)


Lade die CSV-Datei...
Klassen in y: [0 1]
Klassenverteilung: (array([0, 1], dtype=int64), array([29126, 29126], dtype=int64))
Diskretisiere numerische Features (quantile)...


c:\Users\jakob\anaconda3\Lib\site-packages\sklearn\preprocessing\_discretization.py:279: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
c:\Users\jakob\anaconda3\Lib\site-packages\sklearn\preprocessing\_discretization.py:279: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 1 are removed. Consider decreasing the number of bins.
  warnings.warn(
c:\Users\jakob\anaconda3\Lib\site-packages\sklearn\preprocessing\_discretization.py:279: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 9 are removed. Consider decreasing the number of bins.
  warnings.warn(
c:\Users\jakob\anaconda3\Lib\site-packages\sklearn\preprocessing\_discretization.py:279: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 10 are removed. Consider decreasing the number of bins.
  warnings.warn(
c:\Users\jakob\anaconda3\Lib\site-packages\sklearn\preprocessing\_discr


Starte Experiment 1/10 mit random_seed=0...
Trainingszeit: 169.2521 s, Evaluierungszeit: 0.1229 s
Accuracy: 0.5943, Precision: 0.6969, Recall: 0.3336, F1-Score: 0.4512, ROC AUC: 0.5943

Starte Experiment 2/10 mit random_seed=1...


KeyboardInterrupt: 

In [11]:
import pandas as pd
import numpy as np
import time
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.preprocessing import LabelEncoder, KBinsDiscretizer
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    roc_auc_score,
    matthews_corrcoef,
    mean_absolute_error,
    mean_squared_error,
)
from sklearn.model_selection import train_test_split


# Datei laden
print("Lade die CSV-Datei...")
csv_file_path = "C:\\Users\\jakob\\Documents\\Praktikum\\ML_Praktikum\\ml_praktikum_jagoetz_wkathari\\dataset\\clf_num\\Higgs.csv"
data = pd.read_csv(csv_file_path)

# Zielvariable setzen (Spalte: "y")
target_column = "target"

if target_column not in data.columns:
    raise ValueError(f"Die Datei enthält keine '{target_column}'-Spalte. Bitte überprüfen.")

# Features und Zielvariable trennen
X = data.drop(columns=[target_column])
y = data[target_column]

# Zielvariable in nominal umwandeln (falls numerisch)
if y.dtype != 'object':
    print(f"Konvertiere Zielvariable '{target_column}' in nominale Werte...")
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)

# Diskretisierung der numerischen Features
print("Diskretisiere numerische Features...")
discretizer = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')
X = pd.DataFrame(discretizer.fit_transform(X), columns=X.columns)

# Positive Klasse definieren
pos_class = 1  # Annahme: 1 ist die positive Klasse

# Anzahl der Wiederholungen
n_experiments = 10

# Matrizen für Ergebnisse
train_times = []
eval_times = []
accuracies = []
precisions = []
recalls = []
f1_scores = []
roc_aucs = []
mccs = []
maes = []
rmses = []

for i in range(n_experiments):
    print(f"\nStarte Experiment {i + 1}/{n_experiments} mit random_seed={i}...")

    # Train-Test-Split mit zufälligem Seed
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=i)

    # PART (Entscheidungsbaum mit Regelableitung)
    part_model = DecisionTreeClassifier(
        criterion="gini",  # Ähnlich wie PART: Gini-Index für Teilung
        max_depth=None,    # Keine Begrenzung der Tiefe
        min_samples_split=2,  # Minimaler Split
        random_state=i
    )

    # **Trainingszeit messen**
    start_time_train = time.time()
    part_model.fit(X_train, y_train)
    end_time_train = time.time()
    train_time = end_time_train - start_time_train

    # **Vorhersagen und Evaluierung**
    start_time_eval = time.time()
    y_pred = part_model.predict(X_test)
    end_time_eval = time.time()
    eval_time = end_time_eval - start_time_eval

    # Berechnung der Metriken
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    mcc = matthews_corrcoef(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    roc_auc = roc_auc_score(y_test, y_pred) if len(np.unique(y_test)) == 2 else None

    # Ergebnisse speichern
    train_times.append(train_time)
    eval_times.append(eval_time)
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)
    mccs.append(mcc)
    maes.append(mae)
    rmses.append(rmse)
    roc_aucs.append(roc_auc if roc_auc is not None else None)

    # Ergebnisse dieses Experiments ausgeben
    print(f"Trainingszeit: {train_time:.4f} s, Evaluierungszeit: {eval_time:.4f} s")
    print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")

    # Regeln des Entscheidungsbaums ausgeben (optional)
    rules = export_text(part_model, feature_names=list(X.columns))
    print("\nRegeln des Entscheidungsbaums:")
    print(rules)

# **Matrizen erstellen**
train_times_matrix = np.array(train_times).reshape(1, -1)
eval_times_matrix = np.array(eval_times).reshape(1, -1)
metrics_matrix = {
    "Accuracy": np.array(accuracies).reshape(1, -1),
    "Precision": np.array(precisions).reshape(1, -1),
    "Recall": np.array(recalls).reshape(1, -1),
    "F1-Score": np.array(f1_scores).reshape(1, -1),
    "MCC": np.array(mccs).reshape(1, -1),
    "MAE": np.array(maes).reshape(1, -1),
    "RMSE": np.array(rmses).reshape(1, -1),
    "ROC AUC": np.array(roc_aucs).reshape(1, -1),
}

# **Ergebnisse ausgeben**
print("\n=== Ergebnis-Matrizen ===")
print("\nTrainingszeiten (s):")
print(train_times_matrix)

print("\nEvaluierungszeiten (s):")
print(eval_times_matrix)

print("\nMetriken:")
for metric_name, matrix in metrics_matrix.items():
    print(f"\n{metric_name}:")
    print(matrix)


Lade die CSV-Datei...
Konvertiere Zielvariable 'target' in nominale Werte...
Diskretisiere numerische Features...

Starte Experiment 1/10 mit random_seed=0...
Trainingszeit: 9.3809 s, Evaluierungszeit: 0.2590 s
Accuracy: 0.5626, Precision: 0.5619, Recall: 0.5627, F1-Score: 0.5623

Regeln des Entscheidungsbaums:
|--- m_bb <= 0.50
|   |--- jet_1_pt <= 0.50
|   |   |--- jet_2_pt <= 0.50
|   |   |   |--- m_wwbb <= 0.50
|   |   |   |   |--- jet_4_pt <= 0.50
|   |   |   |   |   |--- jet_3_pt <= 0.50
|   |   |   |   |   |   |--- missing_energy_magnitude <= 0.50
|   |   |   |   |   |   |   |--- lepton_pT <= 2.50
|   |   |   |   |   |   |   |   |--- lepton_pT <= 1.50
|   |   |   |   |   |   |   |   |   |--- lepton_pT <= 0.50
|   |   |   |   |   |   |   |   |   |   |--- m_jlv <= 0.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 37
|   |   |   |   |   |   |   |   |   |   |--- m_jlv >  0.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 20
